In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from pymoo.factory import get_performance_indicator
from itertools import product
from palettable.scientific.diverging import Vik_7

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format',lambda x : '%.4f' % x)
plt.style.use(['science','ieee'])


In [ ]:
results_list = [
    "results-database-20220202001809.json"
]

from tinydb import TinyDB, Query

dataset = pd.DataFrame()

for results in results_list:
    database = TinyDB("results/" + results)
    q = Query()
    temp_results = database.all()
    [
        result.update({
            "policy":
            "(s=" + str(int(result['s'])) + ")" if result['algorithm'] == "Heuristic"
            or result['algorithm'] == "HeuristicT"
            or result['algorithm'] == "Augmecon"
            or result['algorithm'] == "WeightedSum" else
            "(nfe=" + str(int(result['nfe'])) + ")",
            "paradigms":str(result['paradigms'])
        }) for result in temp_results
    ]
    temp = pd.DataFrame(temp_results)
    dataset = pd.concat([dataset, temp])
dataset = dataset[[
    "algorithm","paradigms","s","nfe", "run","system_accuracies", "points", "constraints", "experiment_time","policy"
]]
dataset = dataset.fillna(0)
dataset[['s','nfe',"run"]] = dataset[['s', 'nfe',"run"]].astype(int)

refs = pd.DataFrame()
refs['accuracy'] = dataset['constraints'].map(lambda x: x['accuracy'][0])
refs['number'] = dataset['constraints'].map(lambda x: x['number'][0])
refs['bandwidth'] = dataset['constraints'].map(lambda x: x['bandwidth'][0])
ref = refs.min().tolist()
dataset = dataset.drop(columns=["constraints"])

hv = get_performance_indicator("hv", ref_point=np.array(ref) * (-1))
ref


In [ ]:
time_results = dataset[['policy','paradigms','experiment_time','algorithm','run']]

# time_results.sort_values(['experiment_time'], ascending=[False])
time_results.reset_index(inplace=True)
time_results = time_results.rename(columns={"experiment_time": "Time"})
time_results

In [ ]:
df=dataset.copy(deep=True)
heuristic_dataset=[]
if len(df.index)>0:
    df = df[['points', 'policy','paradigms','algorithm','run']]
    df=list(df.to_dict('records'))
    for item in df:
        points=item.pop('points')
        for point in points:
            point.update(item)
            heuristic_dataset.append(point)

df=pd.DataFrame(heuristic_dataset).drop_duplicates(subset=None, keep='first', inplace=False)
# df.head()
hv_results = []
for algorithm in df['algorithm'].unique():
    filter0= df['algorithm']==algorithm
    temp_df=df.where(filter0)
    for policy, paradigms, run in product(temp_df['policy'].unique(),
                                          temp_df['paradigms'].unique(),
                                          temp_df['run'].unique()):
        filter1 = temp_df['policy'] == policy
        filter2 = temp_df['paradigms'] == paradigms
        filter3 = temp_df['run'] == run
        temp = temp_df[[
            'served', 'average_accuracy', 'bandwidth_reserved', 'policy',
            'paradigms', 'algorithm','run'
        ]].where(filter1 & filter2 & filter3).dropna(axis=0, how='any')
        if temp.size > 0:
            data = temp[['average_accuracy', 'served', 'bandwidth_reserved']]
            result = {'policy':policy,'paradigms':paradigms,'algorithm':algorithm,'run':run}
            front = data.values
            result.update({"HV": hv.do(-front)})
            hv_results.append(result)
hv_results = pd.DataFrame(hv_results)
hv_results[["run"]] = hv_results[["run"]].astype(int)
hv_results

In [ ]:
fig, axes = plt.subplots(2,
                         5,
                         gridspec_kw={
                             'width_ratios': [1, 1, 1, 1, 1],
                            #  'height_ratios': [3, 2],
                             "wspace": 0.1,
                             "hspace": 0.1
                         },
                         sharex=False,
                         sharey=False,
                         figsize=(10, 4))

ax1 = sns.barplot(x='policy',
                  y='HV',
                  hue='paradigms',
                  data=hv_results.query('algorithm=="Augmecon"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[0, 1])
# ax1.set_yscale("log")
ax1.set_ylim(200, 1160)
ax1.get_legend().set_visible(False)
ax1.set(xticklabels=[])
ax1.set_title("\\textsf{ECON}", fontsize="medium")
ax1.set(yticklabels=[])
ax1.set_xlabel(None)
ax1.set_ylabel(None)

ax2 = sns.barplot(x='policy',
                  y='HV',
                  hue='paradigms',
                  data=hv_results.query('algorithm=="WeightedSum"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[0, 2])
# ax2.set_yscale("log")
ax2.set_ylim(200, 1160)
ax2.get_legend().set_visible(False)
ax2.set_ylabel(None)
ax2.set(xticklabels=[])
ax2.set_title("\\textsf{WSUM}", fontsize="medium")
ax2.set(yticklabels=[])
ax2.set_xlabel(None)

ax3 = sns.barplot(x='policy',
                  y='HV',
                  hue='paradigms',
                  data=hv_results.query('algorithm=="HeuristicT"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[0, 3])
# ax3.set_yscale("log")
ax3.set_ylim(200, 1160)
ax3.get_legend().set_visible(False)
ax3.set_ylabel(None)
ax3.set(xticklabels=[])
ax3.set_title("\\textsf{WCH-T}", fontsize="medium")
ax3.set(yticklabels=[])
ax3.set_xlabel(None)

ax4 = sns.barplot(x='policy',
                  y='HV',
                  hue='paradigms',
                  data=hv_results.query('algorithm=="Heuristic"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[0, 4])
# ax4.set_yscale("log")
ax4.set_ylim(200, 1160)
ax4.get_legend().set_visible(False)
ax4.set_ylabel(None)
ax4.set(xticklabels=[])
ax4.set_title("\\textsf{WCH}", fontsize="medium")
# ax4.set(yticklabels=[])
ax4.set_xlabel(None)

ax5 = sns.barplot(x='policy',
                  y='HV',
                  hue='paradigms',
                  data=hv_results.query('algorithm=="NSGAII"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[0, 0])
# ax5.set_yscale("log")
ax5.set_ylim(200, 1160)
ax5.get_legend().set_visible(False)
ax5.set_ylabel(None)
ax5.set(xticklabels=[])
ax5.set_title("\\textsf{EVOL}", fontsize="medium")
ax5.set(yticklabels=[])
ax5.set_xlabel(None)

ax6 = sns.barplot(x='policy',
                  y='Time',
                  hue='paradigms',
                  data=time_results.query('algorithm=="Augmecon"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[1, 1])
ax6.set_ylim(10, 5000)
ax6.set_yscale("log")
ax6.get_legend().set_visible(False)
ax6.set(xticklabels=["1000", "5000"])
ax6.set(yticklabels=[])
ax6.set_ylabel(None)

ax7 = sns.barplot(x='policy',
                  y='Time',
                  hue='paradigms',
                  data=time_results.query('algorithm=="WeightedSum"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[1, 2])
ax7.set_ylim(10, 5000)
ax7.set_yscale("log")
ax7.get_legend().set_visible(False)
ax7.set_ylabel(None)
ax7.set(xticklabels=["1000", "5000"])
ax7.set(yticklabels=[])

ax8 = sns.barplot(x='policy',
                  y='Time',
                  hue='paradigms',
                  data=time_results.query('algorithm=="Heuristic"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[1, 3])
ax8.set_ylim(10, 5000)
ax8.set_yscale("log")
ax8.get_legend().set_visible(False)
ax8.set_ylabel(None)
ax8.set(xticklabels=["10000", "50000"])
ax8.set(yticklabels=[])

ax9 = sns.barplot(x='policy',
                  y='Time',
                  hue='paradigms',
                  data=time_results.query('algorithm=="HeuristicT"'),
                  palette=[
                      Vik_7.mpl_colors[1],
                      Vik_7.mpl_colors[3],
                      Vik_7.mpl_colors[5],
                  ],
                  edgecolor=".1",
                  linewidth=1,
                  ax=axes[1, 4])
ax9.set_ylim(10, 5000)
ax9.set_yscale("log")
ax9.get_legend().set_visible(False)
ax9.set_ylabel(None)
ax9.set(xticklabels=["10000", "50000"])
# ax9.set(yticklabels=[])

ax10 = sns.barplot(x='policy',
                   y='Time',
                   hue='paradigms',
                   data=time_results.query('algorithm=="NSGAII"'),
                   palette=[
                       Vik_7.mpl_colors[1],
                       Vik_7.mpl_colors[3],
                       Vik_7.mpl_colors[5],
                   ],
                   edgecolor=".1",
                   linewidth=1,
                   ax=axes[1, 0])
ax10.set_ylim(10, 5000)
ax10.set_yscale("log")
ax10.get_legend().set_visible(False)
ax10.set_ylabel(None)
ax10.set(yticklabels=[])
ax10.set(xticklabels=["10000", "50000"])

ax5.yaxis.label.set_text('\\textbf{ Hypervolume }')
ax10.yaxis.label.set_text('\\textbf{ Time Cost (sec.)}')
ax6.xaxis.label.set_text(r"$\textbf{\# of Weight}$" + "\n" +
                         r"$\textbf{Vectors}$")
ax7.xaxis.label.set_text(r"$\textbf{\# of Weight}$" + "\n" +
                         r"$\textbf{Vectors}$")
ax8.xaxis.label.set_text(r"$\textbf{\# of Weight}$" + "\n" +
                         r"$\textbf{Vectors}$")
ax9.xaxis.label.set_text(r"$\textbf{\# of Weight}$"+"\n"+r"$\textbf{Vectors}$")
ax10.xaxis.label.set_text(r"$\textbf{\# of Function}$" + "\n" +
                          r"$\textbf{Evaluations}$")
ax4.yaxis.tick_right()
ax9.yaxis.tick_right()
ax5.yaxis.set_tick_params(labelsize='small')
ax10.yaxis.set_tick_params(labelsize='small')
ax6.xaxis.set_tick_params(labelsize='small')
ax7.xaxis.set_tick_params(labelsize='small')
ax8.xaxis.set_tick_params(labelsize='small')
ax9.xaxis.set_tick_params(labelsize='small')
ax10.xaxis.set_tick_params(labelsize='small')

for ax in axes.flatten():
    ax.spines['left'].set_linewidth(1)
    ax.spines['right'].set_linewidth(1)
    ax.spines['top'].set_linewidth(1)
    ax.spines['bottom'].set_linewidth(1)
    ax.tick_params(axis='x', which="major", width=1)
    ax.tick_params(axis='y', which="major", width=1)


labels = [
    "\\textbf{SP}",
    "\\textbf{EP+CP}",
    "\\textbf{EP+CP+SP}",
]

handles, _ =ax5.get_legend_handles_labels()
ax5.legend(title=r'\textbf{Deployment}',
          labels=labels,
          handles=handles,
          fontsize='small',
          handlelength=1.5,
          handletextpad=1,
          labelspacing=0.5,
          loc="upper center",
          bbox_to_anchor=(0.5, 0.9)
          )
plt.savefig("figures/deployment.pdf", dpi=300)